# 파일 입력
비전 기능을 갖춘 OpenAI 모델은 PDF 파일을 입력으로 사용할 수도 있습니다. 

모델이 PDF 콘텐츠를 이해할 수 있도록 각 페이지의 추출된 텍스트와 이미지를 모델의 컨텍스트에 추가합니다. 모델은 텍스트와 이미지를 모두 사용하여 응답을 생성할 수 있습니다. 이는 예를 들어 다이어그램에 텍스트에 없는 주요 정보가 포함된 경우 유용합니다.

먼저 Files API를 사용하여 PDF를 업로드한 다음, 모델에 대한 API 요청에서 해당 파일 ID를 참조합니다.

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [2]:
from openai import OpenAI

client = OpenAI()

Model = "gpt-4.1-nano"

In [3]:
# PDF 파일을 OpenAI API에 업로드 (용도: 사용자 데이터로 처리)
file = client.files.create(
    file=open("data/삼성중공업-기업리포트.pdf", "rb"),  # PDF 파일을 바이너리 읽기 모드로 열기
    purpose="user_data"  # 파일 사용 목적 지정 (예: 사용자 데이터 기반 응답 생성)
)

# 업로드된 파일을 기반으로 LLM 응답 생성 요청
response = client.responses.create(
    model=Model,  
    input=[
        {
            "role": "user",  # 사용자 역할로 프롬프트 구성
            "content": [
                {
                    "type": "input_file",       # 파일 입력 유형 지정
                    "file_id": file.id,         # 위에서 업로드한 파일의 고유 ID 참조
                },
                {
                    "type": "input_text",       # 텍스트 입력 유형 지정
                    "text": "파일 내에 있는 첫번째 바차트를 설명해줘.",  # 사용자 지시
                },
            ]
        }
    ]
)

# 모델의 출력 결과를 콘솔에 출력
print(response.output_text)

첫 번째 차트는 전 세계 LNG선(액화천연가스선) 시장의 주요 플레이어들의 수주잔고와 시장 점유율(M/S)을 보여주는 그래프입니다. 

차트의 좌측 축은 수주잔고(척수)이며, 우측 축은 시장 점유율(백분율)을 표시합니다. 그래프에는 여러 선박 제조사들이 나열되어 있는데, 각각의 회사 이름과 해당 선사들이 보유하고 있는 수주잔고와 점유율이 나타나 있습니다. 

이 차트는 각 기업들이 현재 얼마나 많은 LNG선을 수주했는지와 시장 내 점유율 비중을 시각적으로 보여줌으로써, 글로벌 LNG선 시장의 경쟁력을 파악하는 데 도움을 줍니다. 

주요 기업으로는 삼성중공업, 화훼, Hudong Zhonghua, 현중, Dalian Shipbuilding, Zvezda SB, CMHI (Jiangsu), Jiangnan SY Group, Yangzi Xinfu SB, Jiangsu Yixiang SB 등이 포함되어 있습니다.


### Base64로 인코딩된 파일
PDF 파일 입력도 Base64로 인코딩된 입력으로 보낼 수 있습니다.  
base64 encoding은 바이너리 데이터를 텍스트(ASCII) 문자로 안전하게 변환하는 방식입니다.  
보통 이미지, PDF, 동영상, 파일 등 이진 데이터를 문자열로 바꿔서 전송하거나 저장할 때 사용됩니다.

In [4]:
import base64  # 바이너리 데이터를 base64 문자열로 인코딩하기 위한 모듈

# PDF 파일을 바이너리 읽기 모드로 열고 데이터를 읽음
with open("data/삼성중공업-기업리포트.pdf", "rb") as f:
    data = f.read()

# PDF 데이터를 안전한 전송을 위해 base64로 인코딩하고 UTF-8 문자열로 디코딩
base64_string = base64.b64encode(data).decode("utf-8")

# 인코딩된 base64 데이터를 포함하여 모델 응답 생성 요청
response = client.responses.create(
    model=Model,  
    input=[
        {
            "role": "user", 
            "content": [
                {
                    "type": "input_file",      # 파일 입력 형식 명시
                    "filename": "삼성중공업-기업리포트.pdf",  # 파일 이름 지정
                    "file_data": f"data:application/pdf;base64,{base64_string}",  # base64 인코딩된 PDF 데이터
                },
                {
                    "type": "input_text",      # 텍스트 명령 입력
                    "text": "파일 내에 있는 첫번째 바차트를 설명해줘.",  # 사용자 요청 내용
                },
            ],
        },
    ]
)

# 생성된 응답 출력
print(response.output_text)

파일 내에 있는 첫 번째 바 차트는 전 세계 LNG선(액화천연가스선) 시장에서 각 선박 제작사의 수주잔고와 시장 점유율(M/S, Market Share)을 보여주고 있습니다. 

- 좌측 y축은 수주잔고(척수, 선박 수)를 나타내며, 차트의 막대그래프로 표현되어 있습니다. 
- 우측 y축은 각 제조사의 시장 점유율(%)를 나타내며, 선 그래프로 표시되어 있습니다.

차트상 주요 내용은 다음과 같습니다:
- 삼성중공업이 수주잔고 기준으로 가장 높은 수치를 기록하고 있으며, 전 세계 LNG선 시장에서 상당한 점유율을 차지하고 있음을 보여줍니다.
- 기타 주요 제조사들도 각각의 수주잔고와 시장 점유율을 가지고 있으며, 여러 업체들이 경쟁하고 있음을 알 수 있습니다.

전체적으로 이 차트는 삼성중공업이 글로벌 LNG선 시장에서 강한 경쟁력을 갖추고 있음을 시각적으로 보여줍니다.
